This page is for the logistic regression analysis

1. add descriptions of the data clean up; 
1. convert text fields into numeric 0/1 fields
1. drop the text fields after adding the 0/1 equivelents


In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [40]:
train = pd.read_csv('titanic_train.csv')

In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [42]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [43]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

In [44]:
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)

In [45]:
# create a title column, then get_dummies
def get_title(col):
    name = col[0]
    if "Miss" in name:
        return "Miss"
    elif "Master" in name:
        return "Master"
    elif "Mrs" in name:
        return "Mrs"
    elif "Mr" in name:
        return "Mr"

In [46]:
title_string = pd.get_dummies(train[['Name']].apply(get_title,axis=1))

In [47]:
title_string.head()

,Master,Miss,Mr,Mrs
0,0,0,1,0
1,0,0,0,1
2,0,1,0,0
3,0,0,0,1
4,0,0,1,0


In [48]:
def ticket_letters(col):
    Ticket = col[0]
    TicketList = Ticket.split(' ')
    if len(TicketList) > 1:
        return TicketList[0].replace('.','').replace('/','')
    else:
        return ''

In [49]:
ticket_string = pd.get_dummies(train[['Ticket']].apply(ticket_letters,axis=1))

In [50]:
ticket_string.head()

,,A4,A5,AS,C,CA,CASOTON,FC,FCC,Fa,...,SOP,SOPP,SOTONO2,SOTONOQ,SP,STONO,STONO2,SWPP,WC,WEP
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
sex = pd.get_dummies(train['Sex'],drop_first=True)

In [52]:
sex.head()

,male
0,1
1,0
2,0
3,0
4,1


In [53]:
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [54]:
embark.head()

,Q,S
0,0,1
1,0,0
2,0,1
3,0,1
4,0,1


now we add the engineered fields and drop the text fields.

In [55]:
train = pd.concat([train,sex,embark,title_string],axis=1)

In [56]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,male,Q,S,Master,Miss,Mr,Mrs
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0,1,0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,0,0,0,0,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1,0,1,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,0,1,0,0,1,0


In [57]:
#train.dropna(inplace=True)
train.drop(['Sex', 'Embarked','Name','Ticket','Cabin','PassengerId'],axis=1,inplace=True)

In [58]:
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S,Master,Miss,Mr,Mrs
0,0,3,22.0,1,0,7.2500,1,0,1,0,0,1,0
1,1,1,38.0,1,0,71.2833,0,0,0,0,0,0,1
2,1,3,26.0,0,0,7.9250,0,0,1,0,1,0,0
3,1,1,35.0,1,0,53.1000,0,0,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,1,0,0,1,0


X is all the input data; y is what we are trying to predict:

In [59]:
X = train.drop('Survived',axis=1)
y = train['Survived']

In [60]:
from sklearn.cross_validation import train_test_split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
logmodel = LogisticRegression()

In [64]:
train.replace(-np.inf, np.nan)
col_mask=train.isnull().any(axis=0)

In [65]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
male        891 non-null uint8
Q           891 non-null uint8
S           891 non-null uint8
Master      891 non-null uint8
Miss        891 non-null uint8
Mr          891 non-null uint8
Mrs         891 non-null uint8
dtypes: float64(2), int64(4), uint8(7)
memory usage: 47.9 KB


In [66]:
logmodel.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [67]:
predictions = logmodel.predict(X_test)

In [79]:
predictions

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0])

In [68]:
from sklearn.metrics import classification_report, confusion_matrix

In [74]:
print('Confusion Matrix')
print(confusion_matrix(y_test,predictions))
print()
print('Classification Report')
print(classification_report(y_test,predictions))


Confusion Matrix
[[153  16]
 [ 37  89]]

Classification Report
             precision    recall  f1-score   support

          0       0.81      0.91      0.85       169
          1       0.85      0.71      0.77       126

avg / total       0.82      0.82      0.82       295

